In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
df = pd.read_csv('./Data/data_for_reccommender_1.csv')

In [5]:
df.head()

,rest_cost,rest_name,rest_rating,11,11th,12,13,14,14th,15,...,cuisine_type_Spanish,cuisine_type_Steaks,cuisine_type_Sushi,cuisine_type_Tacos,cuisine_type_Taiwanese,cuisine_type_Tex-Mex,cuisine_type_Thai,cuisine_type_Vegetarian,cuisine_type_Vietnamese,cuisine_type_Wine Bar
0,4.0,Omakase Room By Tatsu,7.7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4.0,Sushi Azabu,8.5,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3.0,Saint Julivert Fisherie,7.7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2.0,Farida,8.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2.0,U-Gu,7.7,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [18]:
df.shape

(798, 3848)

## Creating an Item Based Recommender System

In [10]:
cosine_sim = cosine_similarity(df.drop('rest_name', axis = 1), df.drop('rest_name', axis = 1))

In [12]:
indices = pd.Series(df.index)

In [22]:
df[df['rest_name'] == 'U-Gu'].index[0]

4

In [56]:
matches = pd.Series(cosine_sim[4]).sort_values(ascending = False)

In [57]:
top_5 = matches[1:6]

In [58]:
top_5

119    0.529630
330    0.517786
42     0.515992
1      0.510306
71     0.508832
dtype: float64

In [68]:
print(f'{round(top_5.values[0], 2)*100}%')

53.0%


In [64]:
top_5_list = list(top_5.index)

In [67]:
top_5_list

[119, 330, 42, 1, 71]

In [69]:
for i in top_5_list:
    print(f'{df.rest_name[i]} , rating = {df.rest_cost[i]}, cost = {df.rest_cost[i]}')

The Loyal , rating = 3.0, cost = 3.0
Noreetuh , rating = 3.0, cost = 3.0
Beebe’s , rating = 2.0, cost = 2.0
Sushi Azabu , rating = 4.0, cost = 4.0
Abuqir , rating = 1.0, cost = 1.0


In [88]:
def recommendations(name, cosine_sim = cosine_sim):
    #empty list of restaurant
    recommended_restaurants = []
    
    #find index of restaurant that matches the name
    rest_index = df[df['rest_name'] == name].index[0]
    
    #Find the index in the cosine matrix
    matching_index = pd.Series(cosine_sim[rest_index]).sort_values(ascending = False)
    
    #find top 5
    similar_indices = list(matching_index[1:6].index)
    
    #Print top 5 recs
    for i in similar_indices:
        recommended_restaurants.append(df.rest_name[i])
        print(f'{df.rest_name[i]} , rating = {df.rest_rating[i]}, cost = {df.rest_cost[i]}')

In [89]:
recommendations('The Polynesian')

Bar Beau , rating = 7.4, cost = 2.0
The Happiest Hour , rating = 7.5, cost = 2.0
The Spaniard , rating = 6.8, cost = 2.0
The Loyal , rating = 8.4, cost = 3.0
2nd Floor Bar & Essen , rating = 7.9, cost = 3.0


In [90]:
df[df['rest_name'] == 'Wei Williamsburg']

,rest_cost,rest_name,rest_rating,11,11th,12,13,14,14th,15,...,cuisine_type_Spanish,cuisine_type_Steaks,cuisine_type_Sushi,cuisine_type_Tacos,cuisine_type_Taiwanese,cuisine_type_Tex-Mex,cuisine_type_Thai,cuisine_type_Vegetarian,cuisine_type_Vietnamese,cuisine_type_Wine Bar


In [91]:
recommendations('Meadowsweet')

Mayfield , rating = 8.1, cost = 2.0
Marlow & Sons , rating = 8.9, cost = 3.0
Raoul’s , rating = 8.4, cost = 4.0
Birds Of A Feather , rating = 7.8, cost = 2.0
Bar Bolinas , rating = 7.6, cost = 3.0


In [92]:
recommendations('The Modern')

Eleven Madison Park , rating = 9.2, cost = 4.0
The NoMad Bar , rating = 7.8, cost = 3.0
Fishtag , rating = 7.0, cost = 3.0
The Clocktower , rating = 8.2, cost = 4.0
The NoMad , rating = 7.8, cost = 3.0


In [93]:
recommendations('Aita')

Rosemary’s , rating = 7.1, cost = 3.0
Bar Sardine , rating = 8.0, cost = 2.0
Samurai Mama , rating = 7.1, cost = 2.0
The NoMad Bar , rating = 7.8, cost = 3.0
Bar Primi , rating = 7.7, cost = 2.0


In [94]:
recommendations("Chavela's")

Bar San Miguel , rating = 7.6, cost = 2.0
Upland , rating = 9.1, cost = 3.0
Casa Enrique , rating = 9.0, cost = 3.0
Carbone , rating = 9.4, cost = 4.0
Cookshop , rating = 8.4, cost = 3.0


In [95]:
recommendations('Mayfield')

The Loyal , rating = 8.4, cost = 3.0
Simon & The Whale , rating = 8.3, cost = 3.0
Meadowsweet , rating = 8.4, cost = 3.0
Birds Of A Feather , rating = 7.8, cost = 2.0
Fedora , rating = 8.5, cost = 3.0


In [96]:
recommendations('Miss Ada')

Mayfield , rating = 8.1, cost = 2.0
Miss Lily’s , rating = 7.8, cost = 3.0
Cookshop , rating = 8.4, cost = 3.0
Bar Sardine , rating = 8.0, cost = 2.0
Chez Ma Tante , rating = 8.5, cost = 2.0


In [98]:
recommendations('Traif')

Fishtag , rating = 7.0, cost = 3.0
Crispo , rating = 7.7, cost = 3.0
Milos , rating = 8.1, cost = 4.0
Beyoglu , rating = 7.7, cost = 2.0
Balthazar , rating = 8.1, cost = 4.0


In [101]:
df.sample()

,rest_cost,rest_name,rest_rating,11,11th,12,13,14,14th,15,...,cuisine_type_Spanish,cuisine_type_Steaks,cuisine_type_Sushi,cuisine_type_Tacos,cuisine_type_Taiwanese,cuisine_type_Tex-Mex,cuisine_type_Thai,cuisine_type_Vegetarian,cuisine_type_Vietnamese,cuisine_type_Wine Bar
177,2.0,Anella,7.9,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
recommendations('Fiat Café')

Beyoglu , rating = 7.7, cost = 2.0
Sable's , rating = 8.0, cost = 2.0
Sao Mai , rating = 8.3, cost = 1.0
Blondies , rating = 8.0, cost = 1.0
Neil's Coffee Shop , rating = 7.3, cost = 2.0


In [102]:
recommendations('Xixa')

Upland , rating = 9.1, cost = 3.0
Gran Electrica , rating = 8.1, cost = 3.0
Dudley's , rating = 7.1, cost = 3.0
Covina , rating = 8.2, cost = 3.0
Añejo , rating = 7.0, cost = 3.0
